<a href="https://colab.research.google.com/github/jpcloud2021/pyspark-exm/blob/main/json_db_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 67kB/s 
     |████████████████████████████████| 204kB 18.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=55387d468d53af12a4293ad6ae41eadf9ed38568508a2d794eeaefa7a03ecbcd
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [23]:
#test_emp.json
data=[{
   "data": {
       "emp_id": "12345",
       "emp_name": "Mohan",
       "awards": [
           {
               "award_type": "Internal-1",
               "award_name": "Best_emp_of_the_year_1",
               "year": "2001"
           },
           {
               "award_type": "Internal-2",
               "award_name": "Best_emp_of_the_year_2",
               "year": "2002"
           },
           {
               "award_type": "Internal-3",
               "award_name": "Best_emp_of_the_year_3",
               "year": "2003"
           },
           {
               "award_type": "External",
               "award_name": "Best_presenter",
               "year": "2001"
           }
       ]
   }
}]

#main.py
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StringType, StructType
 
schema = StructType() \
             .add("data", StructType() \
                   .add("emp_id", StringType(),True) \
                   .add("emp_name", StringType(), True) \
                   .add("awards", ArrayType(StructType()                     
                       .add("award_type", StringType(), True) \
                       .add("award_name", StringType(), True))) \
               , True)
 


In [24]:
spark= SparkSession.builder\
.appName("MyJsonProcess")\
.master("local[*]")\
.getOrCreate()

spark
 
df1 = spark.createDataFrame(data=data,schema=schema)
df1.show(truncate=False)
df1.printSchema()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|data                                                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{12345, Mohan, [{Internal-1, Best_emp_of_the_year_1}, {Internal-2, Best_emp_of_the_year_2}, {Internal-3, Best_emp_of_the_year_3}, {External, Best_presenter}]}|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+

root
 |-- data: struct (nullable = true)
 |    |-- emp_id: string (nullable = true)
 |    |-- emp_name: string (nullable = true)
 |    |-- awards: array (nullable = true)
 |    |    |-- element:

In [34]:
#df1.select(df1.emp_id).filter(df1.emp_id ==12345).show()
df1.select("data.emp_name").show()
df1.select("data.awards.award_name").show()

+--------+
|emp_name|
+--------+
|   Mohan|
+--------+

+--------------------+
|          award_name|
+--------------------+
|[Best_emp_of_the_...|
+--------------------+



In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import explode

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
json_data = '{"results":[{"a":1,"b":2,"c":"name"},{"a":2,"b":5,"c":"foo"}]}'
json_rdd = sc.parallelize([json_data])
df = spark.read.json(json_rdd)
df =df.withColumn("results", explode(df.results)).select( 
                         col("results.a").alias("a"),
                         col("results.b").alias("b"),
                         col("results.c").alias("c") ) 
df.show()

+---+---+----+
|  a|  b|   c|
+---+---+----+
|  1|  2|name|
|  2|  5| foo|
+---+---+----+



In [42]:
df.filter(df.c =='name').show()

+---+---+----+
|  a|  b|   c|
+---+---+----+
|  1|  2|name|
+---+---+----+



In [47]:
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)
#Then Create DataFrame

swimmersJSON = spark.read.json(stringJSONRDD)
swimmersJSON.printSchema()
swimmersJSON.show()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [51]:
swimmersJSON.where(col('name').like("%a%")).show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
+---+--------+---+-------+



In [52]:
swimmersJSON.where(col('name').like("_a%")).show()

+---+--------+---+-----+
|age|eyeColor| id| name|
+---+--------+---+-----+
| 19|   brown|123|Katie|
+---+--------+---+-----+



In [53]:
swimmersJSON.where(col('name').like("___i%")).show()

+---+--------+---+-----+
|age|eyeColor| id| name|
+---+--------+---+-----+
| 19|   brown|123|Katie|
+---+--------+---+-----+

